In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.13197997, 0.17350962, 0.13131476, 0.04835446, 0.01734802,
       0.01366412, 0.01525642, 0.01188621, 0.02172838, 0.02438666,
       0.0125433 , 0.01157733, 0.01014242, 0.01750169, 0.02699026,
       0.02513979, 0.02215921, 0.01298688, 0.01283184, 0.01404258,
       0.01077666, 0.00951357, 0.01476174, 0.00896167, 0.00992589,
       0.00826245, 0.03178462, 0.00978084, 0.00951568, 0.01821331,
       0.01640968, 0.01053082, 0.01142108, 0.01366569, 0.00863433,
       0.01103228, 0.00923088, 0.01113753, 0.01039721, 0.01070015])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.173510
koi_fpflag_nt        0.131980
koi_fpflag_co        0.131315
koi_fpflag_ec        0.048354
koi_model_snr        0.031785
koi_duration_err1    0.026990
koi_duration_err2    0.025140
koi_time0bk_err2     0.024387
koi_depth            0.022159
koi_time0bk_err1     0.021728
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [8]:
X

,koi_fpflag_ss,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ec,koi_model_snr,koi_duration_err1,koi_duration_err2,koi_time0bk_err2,koi_depth,koi_time0bk_err1
0,0,0,0,0,25.8,0.11600,-0.11600,-0.003520,874.8,0.003520
1,1,0,0,0,76.3,0.03410,-0.03410,-0.000581,10829.0,0.000581
2,1,0,0,0,505.6,0.00537,-0.00537,-0.000115,8079.2,0.000115
3,0,0,0,0,40.9,0.04200,-0.04200,-0.001130,603.3,0.001130
4,0,0,0,0,40.2,0.06730,-0.06730,-0.001900,686.0,0.001900
...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.4,0.63400,-0.63400,-0.015700,87.7,0.015700
6987,1,0,1,0,453.3,0.01740,-0.01740,-0.000170,1579.2,0.000170
6988,0,0,0,0,10.6,0.22900,-0.22900,-0.007690,48.5,0.007690
6989,0,0,1,0,12.3,0.16200,-0.16200,-0.002850,103.6,0.002850


In [9]:
y

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using a logistic regression


In [13]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [14]:
# Score the model
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8447453747854282
Testing Data Score: 0.8443935926773455


# Predict

In [15]:
predictions = model.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
6616,FALSE POSITIVE,FALSE POSITIVE
4291,CONFIRMED,CONFIRMED
3759,CANDIDATE,CANDIDATE
2503,CANDIDATE,CANDIDATE
992,CONFIRMED,CONFIRMED


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [18]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
param_grid = {'C':np.logspace(-4, 4, 20),
             'penalty':['l1','l2']}

grid = GridSearchCV(model, param_grid, verbose=3)

In [19]:
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.0001, penalty=l1 ............................................
[CV] .................. C=0.0001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] .................. C=0.0001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] .................. C=0.0001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] .................. C=0.0001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] .................. C=0.0001, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.00026366508987303583, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.00026366508987303583, penalty=l2 ............................
[CV]  C=0.00026366508987303583, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.00026366508987303583, penalty=l2 ............................
[CV]  C=0.00026366508987303583, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] ... C=0.0006951927961775605, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] ... C=0.0006951927961775605, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] ... C=0.0006951927961775605, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] ... C=0.0006951927961775605, penalty=l1, score=nan, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l1 .............................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    8.6s finished


In [20]:
print(best_model.best_params_)
print('Best Score: ', best_model.best_score_)

{'C': 1438.44988828766, 'penalty': 'l2'}
Best Score:  0.8594328295213908


In [21]:
# Make predictions
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
6616,FALSE POSITIVE,FALSE POSITIVE
4291,CONFIRMED,CONFIRMED
3759,CANDIDATE,CANDIDATE
2503,CANDIDATE,CANDIDATE
992,CONFIRMED,CONFIRMED


In [22]:
# Score the model
best_model.score(X_test_scaled, y_test)

0.8546910755148741